## <center>This Model will Attempt to Predict the Type of Bicycle Used from Trip Data</center>

In [1]:
# Eliminating Warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importing Libraries
import pandas as pd
import numpy as np
import os
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import SMOTE
from sklearn.metrics import balanced_accuracy_score

In [3]:
# Set up data path
data_path = 'data'

In [4]:
# Load base data to dataframe
all_data_file = 'all_data.csv'
all_df = pd.read_csv(os.path.join(data_path, all_data_file))
all_df

,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,6/27/20,Wards Island,13.80,1.0,49.0,1.82,7.60,Specialized
1,10/19/18,CP BL,7.44,NaN,52.0,0.87,8.58,Specialized
2,10/6/16,Citibike to Work,1.27,NaN,10.0,0.17,7.62,Citibike
3,6/30/16,Citibike from Work,1.11,NaN,10.0,0.17,6.66,Citibike
4,5/5/21,Dn UWS Up ES,21.25,2.0,31.0,2.52,8.44,Specialized
...,...,...,...,...,...,...,...,...
362,9/20/16,Citibike to Work,1.09,NaN,12.0,0.20,5.45,Citibike
363,11/17/16,Citibike from Work,0.79,NaN,7.0,0.12,6.77,Citibike
364,6/7/20,To From Astoria,8.25,2.0,1.0,2.02,4.09,Specialized
365,6/21/16,Citibike from Work,0.78,NaN,7.0,0.12,6.69,Citibike


In [5]:
# Change NaN values to 0 - only occurs on hours column so this is accurate
all_df['Hours'] = all_df['Hours'].fillna(0)
all_df

,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,6/27/20,Wards Island,13.80,1.0,49.0,1.82,7.60,Specialized
1,10/19/18,CP BL,7.44,0.0,52.0,0.87,8.58,Specialized
2,10/6/16,Citibike to Work,1.27,0.0,10.0,0.17,7.62,Citibike
3,6/30/16,Citibike from Work,1.11,0.0,10.0,0.17,6.66,Citibike
4,5/5/21,Dn UWS Up ES,21.25,2.0,31.0,2.52,8.44,Specialized
...,...,...,...,...,...,...,...,...
362,9/20/16,Citibike to Work,1.09,0.0,12.0,0.20,5.45,Citibike
363,11/17/16,Citibike from Work,0.79,0.0,7.0,0.12,6.77,Citibike
364,6/7/20,To From Astoria,8.25,2.0,1.0,2.02,4.09,Specialized
365,6/21/16,Citibike from Work,0.78,0.0,7.0,0.12,6.69,Citibike


In [6]:
# Turning Index into Orig_Index Column
all_df.reset_index(inplace=True)
all_df = all_df.rename(columns = {'index': 'Orig_Index'})
all_df

,Orig_Index,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,0,6/27/20,Wards Island,13.80,1.0,49.0,1.82,7.60,Specialized
1,1,10/19/18,CP BL,7.44,0.0,52.0,0.87,8.58,Specialized
2,2,10/6/16,Citibike to Work,1.27,0.0,10.0,0.17,7.62,Citibike
3,3,6/30/16,Citibike from Work,1.11,0.0,10.0,0.17,6.66,Citibike
4,4,5/5/21,Dn UWS Up ES,21.25,2.0,31.0,2.52,8.44,Specialized
...,...,...,...,...,...,...,...,...,...
362,362,9/20/16,Citibike to Work,1.09,0.0,12.0,0.20,5.45,Citibike
363,363,11/17/16,Citibike from Work,0.79,0.0,7.0,0.12,6.77,Citibike
364,364,6/7/20,To From Astoria,8.25,2.0,1.0,2.02,4.09,Specialized
365,365,6/21/16,Citibike from Work,0.78,0.0,7.0,0.12,6.69,Citibike


In [7]:
# Defining fields to use
columns = ['Miles', 'Duration', 'Speed','Type']
# target = ['Type']

In [8]:
# Create Dataframe with desired columns
bike_df = all_df.loc[:,columns].copy()
bike_df

,Miles,Duration,Speed,Type
0,13.80,1.82,7.60,Specialized
1,7.44,0.87,8.58,Specialized
2,1.27,0.17,7.62,Citibike
3,1.11,0.17,6.66,Citibike
4,21.25,2.52,8.44,Specialized
...,...,...,...,...
362,1.09,0.20,5.45,Citibike
363,0.79,0.12,6.77,Citibike
364,8.25,2.02,4.09,Specialized
365,0.78,0.12,6.69,Citibike


In [9]:
# Create Dataframe with desired columns
# Demonstrating different way to do it
# bike_df = all_df[['Miles', 'Duration', 'Speed','Type']]
# bike_df

In [10]:
# Splitting out Features and Target
y = bike_df["Type"]
X = bike_df.drop(columns="Type")

In [11]:
# Splitting out Training and Testing data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [12]:
# Checking training data shape
X_train.shape

(275, 3)

In [13]:
# Checking testing data shape
X_test.shape

(92, 3)

In [14]:
# Creating the Logistic Regression Model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', max_iter=200, random_state=1)

In [15]:
# Training the Model
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

In [16]:
# Create Predictions
y_pred = classifier.predict(X_test)

In [17]:
# Get accuracy score
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.9891304347826086


In [18]:
# Convert Predictions to DataFrame
pred_df = pd.DataFrame(y_pred)
pred_df.reset_index()
pred_df = pred_df.rename(columns = {0:'Prediction'})
pred_df

,Prediction
0,Citibike
1,Citibike
2,Specialized
3,Specialized
4,Specialized
...,...
87,Citibike
88,Citibike
89,Citibike
90,Specialized


In [19]:
# Convert Tests to DataFrame
test_df = pd.DataFrame(y_test)
test_df

,Type
20,Citibike
162,Citibike
74,Specialized
76,Specialized
141,Specialized
...,...
147,Citibike
165,Citibike
323,Citibike
290,Specialized


In [20]:
# Making index into a column
test_df.reset_index(inplace=True)
test_df = test_df.rename(columns = {'index': 'Orig_Index'})
test_df

,Orig_Index,Type
0,20,Citibike
1,162,Citibike
2,74,Specialized
3,76,Specialized
4,141,Specialized
...,...,...
87,147,Citibike
88,165,Citibike
89,323,Citibike
90,290,Specialized


In [21]:
# Combining the new test and predition dataframes horizontally for comparison
test_pred_df = pd.concat([test_df, pred_df], axis = 1)
test_pred_df

,Orig_Index,Type,Prediction
0,20,Citibike,Citibike
1,162,Citibike,Citibike
2,74,Specialized,Specialized
3,76,Specialized,Specialized
4,141,Specialized,Specialized
...,...,...,...
87,147,Citibike,Citibike
88,165,Citibike,Citibike
89,323,Citibike,Citibike
90,290,Specialized,Specialized


In [22]:
# Identify records where prediction is wrong
pred_errs_df = test_pred_df.loc[test_pred_df['Type'] != test_pred_df['Prediction']]
pred_errs_df

,Orig_Index,Type,Prediction
15,179,Specialized,Citibike


In [22]:
# Merging pred_errs_df with all_df to see details of errors
err_details_df = pd.merge(pred_errs_df, all_df, on=["Orig_Index", "Orig_Index"])
err_details_df

,Orig_Index,Type_x,Prediction,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type_y
0,117,Specialized,Citibike,7/15/19,Up Down ES,3.74,0.0,33.0,0.55,6.80,Specialized
1,264,Specialized,Citibike,7/28/19,Roosevelt Island,4.25,0.0,39.0,0.65,6.54,Specialized


In [23]:
# Cleaning up err_details_df to eliminate redundancy and improve readability
err_details_df.drop(columns='Type_y', inplace=True)
err_details_df.rename(columns = {'Type_x' : 'Type'}, inplace=True)
err_details_df

,Orig_Index,Type,Prediction,Date,Destination,Miles,Hours,Minutes,Duration,Speed
0,117,Specialized,Citibike,7/15/19,Up Down ES,3.74,0.0,33.0,0.55,6.80
1,264,Specialized,Citibike,7/28/19,Roosevelt Island,4.25,0.0,39.0,0.65,6.54
